In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# ---------------- Dataset ----------------
data = np.array([100, 102, 101, 105, 107, 106, 108, 110, 109, 111], dtype=np.float32)
data_min, data_max = data.min(), data.max()
data_norm = (data - data_min) / (data_max - data_min)

X = data_norm[:-1].reshape(1, -1, 1)
y = data_norm[1:].reshape(1, -1, 1)

# ---------------- Custom LSTM ----------------
class CustomLSTM(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        input_dim = input_shape[-1]

        self.Wf = self.add_weight((input_dim + self.units, self.units))
        self.Wi = self.add_weight((input_dim + self.units, self.units))
        self.Wc = self.add_weight((input_dim + self.units, self.units))
        self.Wo = self.add_weight((input_dim + self.units, self.units))

        self.bf = self.add_weight((self.units,))
        self.bi = self.add_weight((self.units,))
        self.bc = self.add_weight((self.units,))
        self.bo = self.add_weight((self.units,))

    def step(self, x_t, states):
        h_prev, c_prev = states
        z = tf.concat([h_prev, x_t], axis=1)

        f = tf.sigmoid(tf.matmul(z, self.Wf) + self.bf)
        i = tf.sigmoid(tf.matmul(z, self.Wi) + self.bi)
        c_hat = tf.tanh(tf.matmul(z, self.Wc) + self.bc)
        c = f * c_prev + i * c_hat
        o = tf.sigmoid(tf.matmul(z, self.Wo) + self.bo)
        h = o * tf.tanh(c)

        return h, [h, c]

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]

        h0 = tf.zeros((batch_size, self.units))
        c0 = tf.zeros((batch_size, self.units))

        _, outputs, _ = K.rnn(self.step, inputs, [h0, c0])
        return outputs

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.units)

# ---------------- Model ----------------
inputs = Input(shape=(X.shape[1], X.shape[2]))
x = CustomLSTM(5)(inputs)
outputs = Dense(1)(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(0.01), loss="mse")

# ---------------- Training ----------------
model.fit(X, y, epochs=500, verbose=0)

# ---------------- Prediction ----------------
pred = model.predict(X, verbose=0)
pred_day_11 = pred[0, -1, 0] * (data_max - data_min) + data_min

print("Predicted closing price for day 11:", pred_day_11)


Predicted closing price for day 11: 110.95909
